In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
np.random.seed(3)
pd.set_option('display.float_format', '{:,.8f}'.format)

In [2]:
# Define target predictors
target_predictors = ['AF', 'AGE_YRS', 'Allergic', 'Anemia', 'Asthma', 'CVD', 'Cancer', 'Dementia',
    'Hyperlipidemia', 'Kidney', 'Migraine', 'Thyroid', 'allergies',
    'anxiety', 'arthritis', 'copd', 'curr_ill', 'depression', 'diabetes',
    'disable', 'hypertension', 'obesity', 'othermeds', 'sex', 'MODERNA', 'PFIZER', 'JANSSEN']
print('Num of target predictors:', len(target_predictors))
target_prediction = 'NUMDAYS'
predictor_top_k = 8

Num of target predictors: 27


In [3]:
# Preprocess data
source_data = pd.read_csv('data21_2.csv', low_memory=False)
print('Column keys in vanilla data:', source_data.keys().values)

# Drop duplicate records of same VAERS_ID
source_data.drop_duplicates(subset='VAERS_ID', keep='first', inplace=True)

# Convert VAX_MANU to boolean properties
source_data['MODERNA'] = (source_data['manu'] == 0).astype('int')
source_data['PFIZER'] = (source_data['manu'] == 2).astype('int')
source_data['JANSSEN'] = (source_data['manu'] == 3).astype('int')

# Keep needed columns
source_data = source_data[target_predictors + [target_prediction]]
source_data.dropna(inplace=True)
source_data.reset_index(drop=True, inplace=True)

# Declare NUMDAYS as int
source_data['NUMDAYS'] = source_data['NUMDAYS'].astype('int')

print('Shape of processed source data:', source_data.shape)

Column keys in vanilla data: ['VAERS_ID' 'RECVDATE' 'STATE' 'AGE_YRS' 'CAGE_YR' 'CAGE_MO' 'RPT_DATE'
 'SYMPTOM_TEXT' 'DIED' 'DATEDIED' 'L_THREAT' 'ER_VISIT' 'HOSPITAL'
 'HOSPDAYS' 'X_STAY' 'RECOVD' 'NUMDAYS' 'LAB_DATA' 'V_ADMINBY' 'V_FUNDBY'
 'HISTORY' 'PRIOR_VAX' 'SPLTTYPE' 'FORM_VERS' 'TODAYS_DATE' 'BIRTH_DEFECT'
 'OFC_VISIT' 'ER_ED_VISIT' 'VAX_TYPE' 'VAX_LOT' 'VAX_DOSE_SERIES'
 'VAX_ROUTE' 'VAX_SITE' 'VAX_NAME' 'SYMPTOM1' 'SYMPTOMVERSION1' 'SYMPTOM2'
 'SYMPTOMVERSION2' 'SYMPTOM3' 'SYMPTOMVERSION3' 'SYMPTOM4'
 'SYMPTOMVERSION4' 'SYMPTOM5' 'SYMPTOMVERSION5' 'date_vax' 'date' 'dur'
 'Allergic' 'diabetes' 'hypertension' 'arthritis' 'Asthma' 'Migraine'
 'copd' 'anxiety' 'obesity' 'depression' 'Thyroid' 'Anemia' 'Dementia'
 'Cancer' 'Kidney' 'Hyperlipidemia' 'CVD' 'AF' 'othermeds' 'curr_ill'
 'sex' 'allergies' 'disable' 'manu']
Shape of processed source data: (242719, 28)


In [4]:
# Show distinct counts of each column (make sure that not too many columns have many distinct values)
print([source_data[predictor].nunique() for predictor in target_predictors])

[2, 98, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [5]:
# Scale the source data (for feature importance analysis)
scaler = StandardScaler()
source_data_scaled = scaler.fit_transform(source_data[target_predictors], source_data[target_prediction])
source_data_scaled = pd.DataFrame(source_data_scaled, columns=target_predictors)
source_data.update(source_data_scaled)

In [6]:
# Shuffle data and slice it into training set and test set
source_data = source_data.sample(frac=1, random_state=1).reset_index(drop=True)

len_train = int(source_data.shape[0] * 0.8)
len_test = source_data.shape[0] - len_train

training_data = source_data.loc[:len_train]
test_data = source_data.loc[len_train:]
print('Shape of training set:', training_data.shape)
print('Shape of test set:', test_data.shape)

Shape of training set: (194176, 28)
Shape of test set: (48544, 28)


In [7]:
# And also slice data into two sets with larger/smaller 'NUMDAYS'
shorter_data = source_data[source_data['NUMDAYS'] <= 3].reset_index(drop=True)
longer_data = source_data[source_data['NUMDAYS'] > 3].reset_index(drop=True)
print('Shape of data with shorter onset:', shorter_data.shape)
print('Shape of data with longer onset:', longer_data.shape)

Shape of data with shorter onset: (189098, 28)
Shape of data with longer onset: (53621, 28)


# Util functions

In [8]:
def get_rmse(real_val, pred_val):
	return mean_squared_error(real_val, pred_val, squared=False)

def get_r_square(real_val, pred_val):
	return r2_score(real_val, pred_val)

def print_top_k_predictors(importances, label, sort_key=lambda x:x):
	series = pd.Series(importances, index=target_predictors).sort_values(key=sort_key, ascending=False)
	print(f'Top {predictor_top_k} predictors of {label}:')
	print(series[:predictor_top_k].to_string())

# Ordinary Least Squares (OLS)

In [9]:
regressor_ols = LinearRegression(n_jobs=-1)
regressor_ols.fit(training_data[target_predictors], training_data[target_prediction])

train_pred = regressor_ols.predict(training_data[target_predictors])
print('Train RMSE:', get_rmse(training_data[target_prediction], train_pred))
print('Train R2:', get_r_square(training_data[target_prediction], train_pred))
test_pred = regressor_ols.predict(test_data[target_predictors])
print('Test RMSE:', get_rmse(test_data[target_prediction], test_pred))
print('Test R2:', get_r_square(test_data[target_prediction], test_pred))

Train RMSE: 5.374767375608965
Train R2: 0.028674292271064106
Test RMSE: 5.346096935430181
Test R2: 0.02942295053668742


In [10]:
regressor_ols.fit(shorter_data[target_predictors], shorter_data[target_prediction])
print_top_k_predictors(regressor_ols.coef_, 'OLS (shorter onset)', sort_key=abs)

print()

regressor_ols.fit(longer_data[target_predictors], longer_data[target_prediction])
print_top_k_predictors(regressor_ols.coef_, 'OLS (longer onset)', sort_key=abs)

Top 8 predictors of OLS (shorter onset):
MODERNA        62,935,542,669.93128967
PFIZER         61,365,793,109.90074921
JANSSEN        34,510,858,562.29518890
othermeds                   0.09241762
AGE_YRS                     0.07885676
disable                     0.03141580
sex                        -0.02588068
hypertension                0.02186027

Top 8 predictors of OLS (longer onset):
MODERNA     1,773,246,461,941.48217773
PFIZER      1,729,017,831,579.20263672
JANSSEN       972,364,029,101.61328125
AGE_YRS                     0.62938461
sex                        -0.48857786
allergies                  -0.36345437
othermeds                  -0.32913157
disable                     0.19659613


# Regularized regression

## Lasso

In [11]:
regressor_lasso = LassoCV(cv=10, n_jobs=-1, random_state=1)
regressor_lasso.fit(training_data[target_predictors], training_data[target_prediction])

train_pred = regressor_lasso.predict(training_data[target_predictors])
print('Train RMSE:', get_rmse(training_data[target_prediction], train_pred))
print('Train R2:', get_r_square(training_data[target_prediction], train_pred))
test_pred = regressor_lasso.predict(test_data[target_predictors])
print('Test RMSE:', get_rmse(test_data[target_prediction], test_pred))
print('Test R2:', get_r_square(test_data[target_prediction], test_pred))

Train RMSE: 5.374896556604944
Train R2: 0.028627600639045236
Test RMSE: 5.34618304628657
Test R2: 0.029391683653565148


In [12]:
regressor_lasso.fit(shorter_data[target_predictors], shorter_data[target_prediction])
print_top_k_predictors(regressor_lasso.coef_, 'Lasso (shorter onset)', sort_key=abs)

print()

regressor_lasso.fit(longer_data[target_predictors], longer_data[target_prediction])
print_top_k_predictors(regressor_lasso.coef_, 'Lasso (longer onset)', sort_key=abs)

Top 8 predictors of Lasso (shorter onset):
othermeds       0.09227068
AGE_YRS         0.07884821
disable         0.03131353
JANSSEN        -0.02964864
sex            -0.02574787
hypertension    0.02140770
allergies      -0.01858419
Asthma         -0.01295869

Top 8 predictors of Lasso (longer onset):
AGE_YRS      0.62525810
sex         -0.48869601
MODERNA     -0.46724611
allergies   -0.36571278
othermeds   -0.32236912
PFIZER       0.24352015
disable      0.19524008
diabetes     0.14031417


## Ridge

In [13]:
regressor_ridge = RidgeCV(cv=10)
regressor_ridge.fit(training_data[target_predictors], training_data[target_prediction])

train_pred = regressor_ridge.predict(training_data[target_predictors])
print('Train RMSE:', get_rmse(training_data[target_prediction], train_pred))
print('Train R2:', get_r_square(training_data[target_prediction], train_pred))
test_pred = regressor_ridge.predict(test_data[target_predictors])
print('Test RMSE:', get_rmse(test_data[target_prediction], test_pred))
print('Test R2:', get_r_square(test_data[target_prediction], test_pred))

Train RMSE: 5.374766771776103
Train R2: 0.028674510519895446
Test RMSE: 5.346064364584228
Test R2: 0.02943477689137075


In [14]:
regressor_ridge.fit(shorter_data[target_predictors], shorter_data[target_prediction])
print_top_k_predictors(regressor_ridge.coef_, 'Ridge (shorter onset)', sort_key=abs)

print()

regressor_ridge.fit(longer_data[target_predictors], longer_data[target_prediction])
print_top_k_predictors(regressor_ridge.coef_, 'Ridge (longer onset)', sort_key=abs)

Top 8 predictors of Ridge (shorter onset):
othermeds       0.09247500
AGE_YRS         0.07883890
disable         0.03143554
JANSSEN        -0.02786043
sex            -0.02583260
hypertension    0.02180147
allergies      -0.01899588
Asthma         -0.01304223

Top 8 predictors of Ridge (longer onset):
AGE_YRS      0.62936106
sex         -0.49076649
allergies   -0.37619793
MODERNA     -0.36669283
PFIZER       0.34492824
othermeds   -0.33015192
disable      0.19701201
diabetes     0.15391272


## ElasticNet

*L1 ratio = 0.5*

In [15]:
regressor_elastic = ElasticNetCV(l1_ratio=0.5, cv=10, n_jobs=-1, random_state=1)
regressor_elastic.fit(training_data[target_predictors], training_data[target_prediction])

train_pred = regressor_elastic.predict(training_data[target_predictors])
print('Train RMSE:', get_rmse(training_data[target_prediction], train_pred))
print('Train R2:', get_r_square(training_data[target_prediction], train_pred))
test_pred = regressor_elastic.predict(test_data[target_predictors])
print('Test RMSE:', get_rmse(test_data[target_prediction], test_pred))
print('Test R2:', get_r_square(test_data[target_prediction], test_pred))

Train RMSE: 5.374886596702481
Train R2: 0.02863120062103086
Test RMSE: 5.346190226805657
Test R2: 0.029389076381737156


In [16]:
regressor_elastic.fit(shorter_data[target_predictors], shorter_data[target_prediction])
print_top_k_predictors(regressor_elastic.coef_, 'ElasticNet (shorter onset)', sort_key=abs)

print()

regressor_elastic.fit(longer_data[target_predictors], longer_data[target_prediction])
print_top_k_predictors(regressor_elastic.coef_, 'ElasticNet (longer onset)', sort_key=abs)

Top 8 predictors of ElasticNet (shorter onset):
othermeds       0.09225096
AGE_YRS         0.07884092
disable         0.03131000
JANSSEN        -0.02964651
sex            -0.02574548
hypertension    0.02140351
allergies      -0.01857147
Asthma         -0.01295720

Top 8 predictors of ElasticNet (longer onset):
AGE_YRS      0.62215565
sex         -0.48708481
MODERNA     -0.46171043
allergies   -0.36314240
othermeds   -0.32084079
PFIZER       0.24777225
disable      0.19506148
diabetes     0.13839355


# Random Forest

In [17]:
regressor_rf = RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=1)
regressor_rf.fit(training_data[target_predictors], training_data[target_prediction])

RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=1)

In [18]:
train_pred = regressor_rf.predict(training_data[target_predictors])
print('Train RMSE:', get_rmse(training_data[target_prediction], train_pred))
print('Train R2:', get_r_square(training_data[target_prediction], train_pred))
test_pred = regressor_rf.predict(test_data[target_predictors])
print('Test RMSE:', get_rmse(test_data[target_prediction], test_pred))
print('Test R2:', get_r_square(test_data[target_prediction], test_pred))

Train RMSE: 4.856175844865534
Train R2: 0.20707091681972922
Test RMSE: 5.560120590752197
Test R2: -0.049844025769534506


In [19]:
# Get numerical feature importances
importances = list(regressor_rf.feature_importances_)
print_top_k_predictors(importances, 'RandomForest')

Top 8 predictors of RandomForest:
AGE_YRS        0.36330046
curr_ill       0.05775200
allergies      0.03771437
Thyroid        0.03756740
hypertension   0.03301878
arthritis      0.03299307
MODERNA        0.03236857
othermeds      0.03203971


# Simple Multi-Layer Perceptrons

In [20]:
regressor_nn = MLPRegressor(hidden_layer_sizes=(5, 5), activation='relu', random_state=1)
regressor_nn.fit(training_data[target_predictors], training_data[target_prediction])

MLPRegressor(hidden_layer_sizes=(5, 5), random_state=1)

In [21]:
train_pred = regressor_nn.predict(training_data[target_predictors])
print('Train RMSE:', get_rmse(training_data[target_prediction], train_pred))
print('Train R2:', get_r_square(training_data[target_prediction], train_pred))
test_pred = regressor_nn.predict(test_data[target_predictors])
print('Test RMSE:', get_rmse(test_data[target_prediction], test_pred))
print('Test R2:', get_r_square(test_data[target_prediction], test_pred))

Train RMSE: 5.350419514187296
Train R2: 0.037454630148291645
Test RMSE: 5.329122654570525
Test R2: 0.0355764839125251


In [22]:
coefs = regressor_nn.coefs_[0]
importances = np.sum(np.abs(coefs), axis=1)
print_top_k_predictors(importances, 'MLP')

Top 8 predictors of MLP:
AGE_YRS          4.47443067
othermeds        3.18421053
sex              2.73929950
JANSSEN          2.59034609
Kidney           2.35623525
disable          2.27350989
Migraine         2.11726470
Hyperlipidemia   1.97265987
